In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Qwen2ForCausalLM
from peft import PeftModel
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# 基础模型路径
base_model_path = "models/Qwen2.5-VL-7B-Instruct"
# LoRA适配器路径
lora_adapter_path = "lora_model"


# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    base_model_path, 
    torch_dtype="auto", 
    device_map="auto",
    low_cpu_mem_usage=True
).to('cuda')

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     base_model_path,
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# ).to('cuda')

val_peft_model = PeftModel.from_pretrained(model, lora_adapter_path)

val_peft_model = val_peft_model.half()  # 转换模型权重为FP16

# # default processer
processor = AutoProcessor.from_pretrained(base_model_path, use_fast=True)



D:\Softwares\anaconda\envs\nlp310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 5/5 [00:11<00:00,  2.20s/it]


In [2]:
img_path = "LLaMA-Factory/data/input_image/city_l_安徽_华润_合肥华润中心凯旋门_1梯2户_33层-555.15m²-130-1-1_mix.jpeg"

query = "请根据这张图片中已有的户型信息以及对应的参数，帮我生成其余房间的参数，得到一个完整的合理平面布局。构成户型的所有空间单元均表示为矩形，用x轴坐标、y轴坐标、长度、宽度四个参数表示。本户型为\"城市\"住宅，图片中的为\"一层\"平面。\n图片中已有信息对应的参数为：\n```json\n{\"边界\": [0, 0, 16500, 16200], \"主入口\": [14100, 7500, 1200, 1200], \"南采光\": [-1200, -1200, 17700, 1200], \"北采光\": [-1200, 16200, 17700, 1200], \"西采光\": [-1200, 0, 1200, 16200], \"黑体2\": [12900, 7500, 3600, 8700], \"采光1\": [9300, 15000, 3600, 1200], \"卧室1\": [0, 0, 6000, 7800], \"卧室2\": [0, 10500, 4800, 5700], \"客厅\": [6000, 0, 6900, 6300], \"卧室3\": [4800, 0, 3000, 4500], \"卧室4\": [12900, 0, 3600, 4800], \"厨房\": [6900, 12300, 2400, 3900], \"卫生间\": [4800, 12300, 2100, 3900]}\n```其余待生成的\"一层\"房间的名称为：\n```json\n[\"主卫\", \"储藏\", \"餐厅\"]```"

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": img_path,
            },
            {"type": "text", "text": query},
        ],
    }
]

# # Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to('cuda')

# # Inference: Generation of the output
generated_ids = val_peft_model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

D:\Softwares\anaconda\envs\nlp310\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 40.50 GiB. GPU 0 has a total capacty of 23.99 GiB of which 1.66 GiB is free. Of the allocated memory 20.45 GiB is allocated by PyTorch, and 319.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF